# Raster Mask

This notebook contains analysis and visualizations for:

1. Loading a geotiff file into python 
2. Masking rasters at set thresholds 

Updated: 2018-11-21

### Raster Data to Explore
    Landscan
    Worldpop
    NTL - DMSP & VIIRS
    Africapolic 
    EU Human Settlement Layer
    MODIS built layer
    Facebook Human Settlement
    GRUMP
    
### Define Urban Density
    Go w/ former US Census threshold of >= 1000 ppl per sq mile
    https://www2.census.gov/geo/pdfs/reference/GARM/Ch12GARM.pdf
    
    http://blogs.worldbank.org/sustainablecities/what-does-urban-mean

In [ ]:
import os
import matplotlib.pyplot as plt
import rasterio
from rasterio import plot
from rasterio.mask import mask
import pandas as pd
import geopandas as gpd
import fiona
import rasterio
import numpy as np
import skimage.io as skio
import time
import copy
from shapely.geometry import Polygon

# Step 1 - Input file

    - Becomes a rasterio object, check meta data 
    - Bands are numpy arrays

In [ ]:
# will build out folders later

# data folder git will ignore
infilepath = "/home/cascade/tana-crunch-cascade/projects/NTL/data/" # git will ignore
outfilepath = "/home/cascade/tana-crunch-cascade/projects/NTL/temp_data/" # git will not ignore - NO BIG FILES 

# Local computer 
# infilepath = '/Users/cascade/Github/NTL/data/raw/worldpop/Africa-1km-Population/'
# outfilepath = '/Users/cascade/Github/NTL/temp_data/'

In [ ]:
# Need to build function to loop through files

africa1k_2000 = rasterio.open(infilepath+"AFR_PPP_2000_adj_v2.tif") #<----- changes to 2020 18/13/11 @ 2pm 
print(africa1k_2000.meta)

africa1k_2020 = rasterio.open(infilepath+"AFR_PPP_2020_adj_v2.tif") #<----- changes to 2020 18/13/11 @ 2pm 
print(africa1k_2000.meta)

# Step 2 - Make a mask

In [ ]:
# Set density
# https://www2.census.gov/geo/pdfs/reference/ua/Defining_Rural.pdf

# density = 1000/ 2.59 #density per sq km from sq mi
# print(density)

In [ ]:
# Let's try to make the mask directy with rasterio

def raster_mask(raster, band, density):
    """Fucntion makes numpy array for of raster then applies mask.
    requires rasterio object, band number, and density as float on int 
    returns dict of polygons and numpy mask
    """

    mask = raster.read(band)
    mask[mask < density] = 0 # mask as zeros 
    mask[mask >= density] = 1 
    
    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(mask, transform=raster.transform):
        
        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326).
        geom = rasterio.warp.transform_geom(raster.crs, 'EPSG:4326', geom, precision=6) # WGS84 (EPSG 4326)
    
    # turn geom into Polygon object from shapely    
    Polygon(geom['coordinates'][0])
    poly_gdf = gpd.GeoDataFrame()
    for i, poly in enumerate(geom['coordinates']):
        poly_gdf.loc[i,'geometry']= Polygon(poly)

    return mask, geom, poly_gdf

In [ ]:
mask, geom, poly = raster_mask(africa1k_2000, 1, 600)

### Cell to write out mask as geotiff as needed

In [ ]:
# kwargs = africa1k_2000.meta
# kwargs.update(dtype=rasterio.float32, count = 1)

# with rasterio.open(infilepath+'africa1k_2020_mask600.tif', 'w', **kwargs) as dst:
#         dst.write_band(1, new_mask.astype(rasterio.float32))


### Cell to write out polygons as needed (shape file saves trouble)

In [ ]:
# Need to drop the first row because it's the footprint of the raster
# BE SURE TO MAKE NEW COL NAMES if using CSV

# poly = poly.drop(poly.index[0], axis = 0)

# poly.to_file(outfilepath+'20181121_poly_cities_africa2000mask600.shp', driver='ESRI Shapefile')
# poly.to_csv(outfilepath+"20181121_poly_cities_africa2000mask600.csv")

# OLD CODE

In [ ]:
# from rasterio.features import shapes
# mask = None
# with rasterio.drivers():
#     with rasterio.open(filepath+"GHA10_040213.tif", window = ((7000, 7200), (2700, 3300))) as src:
#         image = src.read(1) # first band
#         results = (
#         {'properties': {'raster_val': v}, 'geometry': s}
#         for i, (s, v) 
#         in enumerate(
#             shapes(image, mask=mask, transform=src.affine)))

In [ ]:
# Sanity check on number of polygons 
# from skimage import measvure
# label_test = measure.label(mask_dilated, background=0, return_num=True)
# label_test_arr = label_test[0]
# label_test_num = label_test[1] # 96 polygons
# label_test_num

In [ ]:
# from shapely.geometry import shape
# import ast

# test_poly = ast.literal_eval(polypoints_2000_df.iloc[0,6])
# print(type(test_poly))

# test = shape(test_poly)
# test

In [ ]:
# test_point = osm_gdf.iloc[4]['geometry']
# test_poly = poly_geo_df.iloc[0]['geometry']
# print(test_poly.contains(test_point))
# print(test_point.within(test_poly))

In [ ]:
# # test if point is inside poly
# checkpoint = time.time()

# test_point = osm_gdf.iloc[30]['geometry']
# test_poly = poly_geo_df.iloc[2]['geometry']
# print(test_poly.contains(test_point))
# print(test_point.within(test_poly))
# print(test_poly)
# print(test_point)

In [ ]:
# Sanity check on number of polygons 
# from skimage import measure
# label_test = measure.label(new_mask, background=0, 
#                            connectivity = 2, # diagonal connect
#                            #neighbors = 8,
#                            return_num=True)
# label_test_arr = label_test[0]
# label_test_num = label_test[1] # 96 polygons

# no connectivity change 38377

# Dilate and Erode 
With 3x3 kernal ... must be odd number to stay centered 

In [ ]:
# With 3x3 kernal ... must be odd number to stay centered  
# try dilate, erode, open and close

# import cv2

# kernal = np.ones((3,3), np.uint8)
# mask_dilated_close = cv2.morphologyEx(new_mask, cv2.MORPH_CLOSE, kernal)
# mask_dilated_open = cv2.morphologyEx(new_mask, cv2.MORPH_OPEN, kernal)
# mask_erode = cv2.erode(new_mask, kernal, iterations = 1)
# mask_dilate = cv2.dilate(new_mask, kernal, iterations = 1)

# write them out and check in QGIS
# ----------------> Need to do this without writing files <-----
# kwargs = africa1k_2000.meta

# Update kwargs (change in data type)
# kwargs.update(dtype=rasterio.float32, count = 1)

# Let's see what is in there
# print(kwargs)

# erode check
# with rasterio.open(infilepath+'africa1k_2020_maskErode.tif', 'w', **kwargs) as dst:
#         dst.write_band(1, mask_erode.astype(rasterio.float32))

# dilate check 
# with rasterio.open(infilepath+'africa1k_2020_maskDilate.tif', 'w', **kwargs) as dst:
#         dst.write_band(1, mask_dilate.astype(rasterio.float32)) 
        
# close raster check
# with rasterio.open(infilepath+'africa1k_2020_maskClose.tif', 'w', **kwargs) as dst:
#         dst.write_band(1, mask_dilated_close.astype(rasterio.float32))

# open raster check        
# with rasterio.open(infilepath+'africa1k_2020_maskOpen.tif', 'w', **kwargs) as dst:
#         dst.write_band(1, mask_dilated_open.astype(rasterio.float32))

#plot.show(mask_dilated)

#### Notes
- Dilate alone connects too many cites
- Erode alone may make too few cities
- Open greatly reduces the number of polygons but may be best for a conservitive estimate
- Close connects them and may make it difficult to attach the correct label with correct political boundaries

In [ ]:
# from rasterio import features as features

# with rasterio.open(infilepath+'africa1k_2020_mask600.tif') as dataset:

#     # Read the dataset's valid data mask as a ndarray.
#     mask = dataset.read(1)

#     # Extract feature shapes and values from the array.
#     for geom, val in rasterio.features.shapes(
#             mask, transform=dataset.transform):

#         # Transform shapes from the dataset's own coordinate
#         # reference system to CRS84 (EPSG:4326).
#         geom = rasterio.warp.transform_geom(
#             dataset.crs, 'EPSG:4326', geom, precision=6) # WGS84 (EPSG 4326)

# print("elapsed time is: {}s".format(time.time()-checkpoint))

In [ ]:
# Turn rasterio geom dict into a polygon

# from shapely.geometry import Polygon

# def build_poly_df(geom):
#     """
#     This function makes polygons from rasterio geom dict
#     """
    
#     # turn dict into normal data frame
#     Polygon(geom['coordinates'][0])
#     geo_df = gpd.GeoDataFrame()
#     for i, poly in enumerate(geom['coordinates']):
#         geo_df.loc[i,'geometry']= Polygon(poly)
#     return geo_df

In [ ]:
# import pandas as pd
# from shapely.geometry import Point

# osm_df = pd.read_csv(outfilepath+'africa_cities.csv')

# # creating a geometry column 
# geometry = [Point(xy) for xy in zip(osm_df['lon'], osm_df['lat'])]

# # Coordinate reference system : WGS84
# crs = {'init': 'epsg:4326'}

# # Creating a Geographic data frame 
# osm_gdf = gpd.GeoDataFrame(osm_df, crs=crs, geometry=geometry)

In [ ]:
# building a function to check if points are in poly for lists of poly and points
# needs geopandas data frame with point and poly geometry 

# from shapely.geometry import mapping

# def poly_point (poly, point):
#     """
#     This function will check if points are inside polygons if given two gpds with points and polygons
#     Returns city names or no list 
#     """
    
#     out_arr = [] #return an array <<< ---------------- ASK RYAN IF BETTER DO USE DICT 
    
#     for index_point, row_point in point.iterrows():
#         for index_poly, row_poly in poly.iterrows():
#             if row_point['geometry'].within(row_poly['geometry']):
#                 country = row_point['Country']
#                 city = row_point['City']
#                 point_id = row_point['Id']
#                 point_geom = mapping(row_point['geometry']) # makes a dict w/ keys : type and cood
#                 poly_id = index_poly
#                 poly_geom = mapping(row_poly['geometry']) # makes a dict w/ keys : type and cood
                
#                 out_arr.append((country, 
#                                 city, 
#                                 point_id, 
#                                 point_geom, 
#                                 poly_id, 
#                                 poly_geom))
# #             else:
# #                 test.append('no')
#     return out_arr

In [ ]:
# # 863 total cities in osm, 26974 polygons took 1578 seconds for 2000
# # 863 total cities in osm, 42241 polygons took XXXX seconds for 2020

# sub_point = osm_gdf.iloc[0:4]
# sub_poly = poly_geo_df_copy.iloc[0:1000] #<----- With copy, drop raster foot print polygon

# # print(sub_point.shape)
# # print(sub_poly.shape)
# # print(sub_point)
# # print(sub_poly)

In [ ]:
# osm_gdf.shape

In [ ]:
# check_point = time.time()

# # (poly then points)
# test = poly_point(poly_geo_df_copy, osm_gdf)
# print(len(test))

# print("elapsed time is: {}s".format(time.time()-check_point))



In [ ]:
# # building a function to check if points are in poly for lists of poly and points
# # needs geopandas data frame with point and poly geometry 

# from shapely.geometry import mapping

# def poly_point (poly, point):
#     """
#     This function will check if points are inside polygons if given two gpds with points and polygons
#     Returns city names or no list 
#     """
    
#     out_arr = [] #return an array <<< ---------------- ASK RYAN IF BETTER DO USE DICT 
    
#     for index_point, row_point in point.iterrows():
#         for index_poly, row_poly in poly.iterrows():
#             if row_point['geometry'].within(row_poly['geometry']):
#                 country = row_point['Country']
#                 city = row_point['City']
#                 point_id = row_point['Id']
#                 point_geom = mapping(row_point['geometry']) # makes a dict w/ keys : type and cood
#                 poly_id = index_poly
#                 poly_geom = mapping(row_poly['geometry']) # makes a dict w/ keys : type and cood
                
#                 out_arr.append((country, 
#                                 city, 
#                                 point_id, 
#                                 point_geom, 
#                                 poly_id, 
#                                 poly_geom))
# #             else:
# #                 test.append('no')
#     return out_arr

In [ ]:
# from shapely.geometry import shape
# import ast

# test_1 = []
# test_2 = []

# for i in polypoints_2020_df['poly_geom']:
#     i = (ast.literal_eval(i))
#     test_1.append((i))
    
# for j in test_1:
#     j = shape(j)
#     test_2.append((j))
    
# # for poly in polypoints_2000_df.iloc[:,6]:
# #     poly = shape(ast.literal_eval(poly))
# #     test.append = (poly)

# #polypoints_2020_df['poly_geom'] = polypoints_2020_df['poly_geom'].apply(ast.literal_eval())
# polypoints_2020_df['geometry'] = test_2

#### When working on home computer not Tana start here 18.11.13

    tasks
    - suck in .csv with cities for 2000 and 2020
    - turn polygons into shape files
    - turn shape files into masks
    - use masks to return raster stats for the polygons

In [ ]:
# # open polypoints csvs

# polypoints_2000_df =  pd.read_csv(outfilepath+'20181113_polypoints_WP2000_africa.csv')
# polypoints_2020_df =  pd.read_csv(outfilepath+'20181113_polypoints_WP2020_africa.csv')

# print(polypoints_2000_df.head(1))
# #print(polypoints_2020_df.head(1))

In [ ]:
# polypoints_2000_df.columns = ['i_val','country','city','osm_id',
#                              'point_geom','poly_id','poly_geom']

# polypoints_2020_df.columns = ['i_val','country','city','osm_id',
#                              'point_geom','poly_id','poly_geom']
# print(polypoints_2000_df.head(1))

In [ ]:
# print(polypoints_2000_df.shape) #712
# print(polypoints_2020_df.shape) #745


In [ ]:
# from shapely.geometry import shape
# import ast

# test_1 = []
# test_2 = []

# for i in polypoints_2020_df['poly_geom']:
#     i = (ast.literal_eval(i))
#     test_1.append((i))
    
# for j in test_1:
#     j = shape(j)
#     test_2.append((j))
    
# # for poly in polypoints_2000_df.iloc[:,6]:
# #     poly = shape(ast.literal_eval(poly))
# #     test.append = (poly)

# #polypoints_2020_df['poly_geom'] = polypoints_2020_df['poly_geom'].apply(ast.literal_eval())
# polypoints_2020_df['geometry'] = test_2

In [ ]:
# polypoints_2020_df.head(1)
# print(polypoints_2020_df.shape)

In [ ]:
# polypoints_2020_gdf = gpd.GeoDataFrame(polypoints_2020_df, geometry='geometry')
# polypoints_2000_gdf = gpd.GeoDataFrame(polypoints_2000_df, geometry='geometry')
# polypoints_2000_gdf.head(1)

In [ ]:
# polypoints_2000_gdf.to_file(outfilepath+'20181113_shape_WP2000_africa.shp', driver='ESRI Shapefile')
# polypoints_2020_gdf.to_file(outfilepath+'20181113_shape_WP2020_africa.shp', driver='ESRI Shapefile')

# test_poly = polypoints_2020_df.iloc[1,7] # this is Oran Algeria w/ wiki population 1.16 million
# test_poly

In [ ]:
# africa1k_2000.meta['transform']

In [ ]:
# from rasterstats import zonal_stats

# africa1k_2000_b1 = africa1k_2000.read(1)
# africa1k_2020_b1 = africa1k_2020.read(1)

# zone_WP2000 = zonal_stats(polypoints_2000_gdf, africa1k_2000_b1, affine=africa1k_2000.meta['transform'], stats = 'sum')
# zone_WP2020 = zonal_stats(polypoints_2020_gdf, africa1k_2020_b1, affine=africa1k_2020.meta['transform'], stats = 'sum')


In [ ]:
# # Need to figure out how to turn geo pandas back into a geojson dict
# with fiona.open("worldpop_2000_test.shp", "r") as shapefile:                                #<--- Need to figure out how to get the geometry from shp file directly 
#     features = [feature["geometry"] for feature in shapefile]

# with rasterio.open(filepath+"AFR_PPP_2000_adj_v2.tif") as src:                        #<--- Need to figure out how to get it not to write to a file, but just store in memory at this point
#     out_image, out_transform = rasterio.mask.mask(src, features, crop=True)
#     out_meta = src.meta.copy()

# out_meta.update({"driver": "GTiff",
#                  "height": out_image.shape[1],
#                  "width": out_image.shape[2],
#                  "transform": out_transform})
# with rasterio.open(filepath+"AFR_PPP_2000_adj_MASK.tif", "w", **out_meta) as dest:
#     dest.write(out_image) 
